In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import os

In [2]:
data = pd.read_csv('data/train/trainx.txt', sep='\t')
features = data.columns.values
print("特征如下：{} \n".format(features))
print("数据数目: ", len(data))

特征如下：['ITEM_NAME' 'TYPE'] 

数据数目:  499447


## 数据清洗
* 替换一些字符 类似于 （ ） ‘ ’ _

In [3]:
import re
from zhon.hanzi import punctuation
from string import punctuation as english_punc
def  rm_punc(strs):
    return re.sub(r"[{}]+".format(punctuation + english_punc)," ",strs)
data[features[0]]  = data[features[0]].apply(rm_punc)
data.head()

,ITEM_NAME,TYPE
0,腾讯QQ黄钻三个月QQ黄钻3个月季卡官方自动充值可查时间可续费,本地生活--游戏充值--QQ充值
1,腾讯q币充值30元qq币30qb30q币自动充值,本地生活--游戏充值--QQ充值
2,腾讯QQ红钻包月卡 QQ红钻一个月QQ红钻1个月,本地生活--游戏充值--QQ充值
3,腾讯QQ蓝钻贵族31个月直充,本地生活--游戏充值--QQ充值
4,腾讯QQ币148元148QQ币148个直充148Q币148个Q币148个QQB★自动充值,本地生活--游戏充值--QQ充值


In [4]:
# 因为有的label里本身就存在空格，所以先要把空格去掉，再去掉- - ， 再以空格划分
data = shuffle(data)
data_y = data[features[1]]

# 取训练数据和测试数据
train = data.sample(frac= 0.8).reset_index()
test = data.sample(frac= 0.2).reset_index()


print("训练数据数目:  ", len(train))
print("测试数据数目:  ", len(test))

train_x  = train[features[0]]
train_y  = train[features[1]]

test_x  = test[features[0]]
test_y  = test[features[1]]

print(train_x.shape, test_x.shape)

# 分出数据集 训练集 测试集 数据集用来训练词向量
data_y = data_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
data_y= [each.split() for each in data_y]

train_y = train_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
train_y = [each.split() for each in train_y]

test_y = test_y.apply(lambda x: x.replace(" ", "").replace("--", " ").replace("/", "")).tolist()
test_y = [each.split() for each in test_y]

训练数据数目:   399558
测试数据数目:   99889
(399558,) (99889,)


## 分词

In [5]:
import jieba
from random import shuffle
def participle(data, doshuffle = False):
    words = []
    for i in range(len(data)):
        result=[]
        seg_list = jieba.cut(data.iloc[i])
        for w in seg_list :#读取每一行分词
            if w != " ":
                result.append(w)
        # 打乱每条数据的分词
        if doshuffle:
            shuffle(result)
        words.append(result)#将该行分词写入列表形式的总分词列表
    return words

In [6]:
# 每条数据的分词结果是个长度不等的词列表
word_data = participle(data[features[0]], doshuffle = False)
word_data_train = participle(train_x,doshuffle = False)
word_data_test = participle(test_x, doshuffle = False)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Richado\AppData\Local\Temp\jieba.cache
Loading model cost 0.727 seconds.
Prefix dict has been built succesfully.


In [7]:
data[features[0]][0]

'腾讯QQ黄钻三个月QQ黄钻3个月季卡官方自动充值可查时间可续费'

In [8]:
from random import shuffle
word_data[8]

['北京', '特产', '零食', '好', '亿家', '豌豆黄', '散装', '零食', '500g']

## 标签分离(一个标签分成三个标签)

In [9]:
train_y = pd.DataFrame(train_y, columns=['label1', 'label2', 'label3'])
test_y = pd.DataFrame(test_y, columns=['label1', 'label2', 'label3'])
data_y = pd.DataFrame(data_y, columns=['label1', 'label2', 'label3'])

In [10]:
len(data_y["label1"].unique())

22

In [11]:
len(data_y["label2"].unique())

191

In [12]:
len(data_y["label3"].unique())

1192

## 标签onn_hot

In [13]:
from keras.utils import to_categorical
# 先生成唯一数组
y_label1 = data_y['label1'].unique().tolist()
y_label2 = data_y['label2'].unique().tolist()
y_label3 = data_y['label3'].unique().tolist()

# 获取在唯一数组中的索引(训练集和测试集各有3个标签需要处理)
train_y_label1_map = train_y['label1'].apply(lambda x: y_label1.index(x))
train_y_label2_map = train_y['label2'].apply(lambda x: y_label2.index(x))
train_y_label3_map = train_y['label3'].apply(lambda x: y_label3.index(x))
test_y_label1_map = test_y['label1'].apply(lambda x: y_label1.index(x))
test_y_label2_map = test_y['label2'].apply(lambda x: y_label2.index(x))
test_y_label3_map = test_y['label3'].apply(lambda x: y_label3.index(x))

Using TensorFlow backend.


In [14]:
# 生成对应one-hot(用做训练模型的标签)
train_y_label1_one_hot = to_categorical(train_y_label1_map, 22)
train_y_label2_one_hot = to_categorical(train_y_label2_map, 191)
train_y_label3_one_hot = to_categorical(train_y_label3_map, 1192)
test_y_label1_one_hot = to_categorical(test_y_label1_map, 22)
test_y_label2_one_hot = to_categorical(test_y_label2_map, 191)
test_y_label3_one_hot = to_categorical(test_y_label3_map, 1192)

In [15]:
from keras.preprocessing.text import *
from keras.preprocessing import sequence
tok = Tokenizer()
tok.fit_on_texts(word_data)

In [16]:
tok.word_index

{'新款': 1,
 '1': 2,
 '汽车': 3,
 '时尚': 4,
 '黑色': 5,
 '鞋': 6,
 '2016': 7,
 '休闲': 8,
 '款': 9,
 '专用': 10,
 '2': 11,
 '5': 12,
 '男士': 13,
 '跟': 14,
 '手机': 15,
 '适用': 16,
 '女': 17,
 '套装': 18,
 '男': 19,
 '韩版': 20,
 '夏季': 21,
 '儿童': 22,
 '3': 23,
 '包': 24,
 '于': 25,
 '透气': 26,
 '壳': 27,
 '套': 28,
 '休闲鞋': 29,
 '米': 30,
 '户外': 31,
 '男鞋': 32,
 '真皮': 33,
 '垫': 34,
 '红色': 35,
 '6': 36,
 '4': 37,
 '脚垫': 38,
 '女鞋': 39,
 '男女': 40,
 '白色': 41,
 '全': 42,
 '蓝色': 43,
 '创意': 44,
 '宝宝': 45,
 '防水': 46,
 '简约': 47,
 '四季': 48,
 '女士': 49,
 '8': 50,
 '保护套': 51,
 '凉鞋': 52,
 '牛皮': 53,
 '四件套': 54,
 '皮鞋': 55,
 '系列': 56,
 '专车': 57,
 '情侣': 58,
 '包围': 59,
 '灯': 60,
 '通用': 61,
 '纯棉': 62,
 '加厚': 63,
 '小': 64,
 '运动': 65,
 '单鞋': 66,
 '商务': 67,
 '学生': 68,
 '大': 69,
 '2015': 70,
 '送': 71,
 '潮流': 72,
 '客厅': 73,
 '婴儿': 74,
 '带': 75,
 '新': 76,
 '卡通': 77,
 '号': 78,
 '全棉': 79,
 '英伦': 80,
 '7': 81,
 '0': 82,
 '寸': 83,
 '可': 84,
 '现代': 85,
 '鞋子': 86,
 'led': 87,
 '单肩': 88,
 '玩具': 89,
 '黑': 90,
 '被': 91,
 '座垫': 92,
 '39': 93,
 '礼物': 94,

In [17]:
len(tok.word_index)

274420

In [18]:
x_train = tok.texts_to_sequences(word_data_train)

In [19]:
x_test = tok.texts_to_sequences(word_data_test)

In [20]:
x_train = sequence.pad_sequences(x_train, maxlen=20)
x_test  = sequence.pad_sequences(x_test, maxlen=20) 

In [21]:
x_test

array([[ 29967,    107,      1, ...,      5,    182,    102],
       [     0,      0,      0, ...,   3281, 147266,    226],
       [     3,     38,     42, ...,     57,     10,    923],
       ...,
       [     0,      0,   4474, ...,   4963,      9,     23],
       [   597,    538,    155, ...,    378,   1109,   1541],
       [     0,      0,      0, ...,    170,   3516,    999]])


## GRU Model

In [22]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import tensorflow.contrib.layers as layers
class GruModel(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    """
    def __init__(self, 
                 sequence_length, 
                 num_classes, 
                 embedding_size,  
                 num_filters, 
                 l2_reg_lambda=0.0, 
                 n_layer = 1, 
                 hidden_size = 32, 
                 batch_size = 256, 
                 vac_size = 27440):
        """
        sequence_length : 一个句子的长度（词的个数）
        embedding_size : 词向量的长度
        num_classes : 三个标签的类别数
        vac_size : 词的个数
        """
        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y1 = tf.placeholder(tf.float32, [None, num_classes[0]], name="input_y1")
        self.input_y2 = tf.placeholder(tf.float32, [None, num_classes[1]], name="input_y2")
        self.input_y3 = tf.placeholder(tf.float32, [None, num_classes[2]], name="input_y3")
        self.keep_prob = tf.placeholder(tf.float32, name="keep_prob")
        self.n_layer = n_layer
        self.batch_size = batch_size
        self.hidden_size = hidden_size

        
        with tf.name_scope("embedding"):
            W = tf.Variable(
                tf.random_uniform([274420, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(W, self.input_x)

        l2_loss = tf.constant(0.0)
        
        #gru模型
        with tf.variable_scope('bigru_text'):
            self.gru_output = self.bigru_inference(self.embedded_chars)
            print("gru shape", self.gru_output.shape)
            
        self.h_drop = self.gru_output
        num_filters_total = self.h_drop.shape[1]
        
        with tf.name_scope("output1"):
            W1 = tf.get_variable(
                "W11",
                shape=[num_filters_total, 64],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[64]), name="b11")
            self.output = tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output1")
            self.output = tf.layers.batch_normalization(self.output, training=is_training)
            self.output = tf.nn.relu(self.output)
            self.output = tf.nn.dropout(self.output, self.keep_prob)
            W2 = tf.get_variable(
                "W12",
                shape=[64, num_classes[0]],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes[0]]), name="b12")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores1 = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions1 = tf.argmax(self.scores1, 1, name="predictions")
            
        with tf.name_scope("output2"):
            W1 = tf.get_variable(
                "W21",
                shape=[num_filters_total, 128],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[128]), name="b21")
            self.output = tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output2")
            self.output = tf.layers.batch_normalization(self.output, training=is_training)
            self.output = tf.nn.relu(self.output)
            self.output = tf.nn.dropout(self.output, self.keep_prob)
            W2 = tf.get_variable(
                "W22",
                shape=[128, num_classes[1]],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes[1]]), name="b22")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores2 = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions2 = tf.argmax(self.scores2, 1, name="predictions")
            
        with tf.name_scope("output3"):
            W1 = tf.get_variable(
                "W31",
                shape=[num_filters_total, 256],
                initializer=tf.contrib.layers.xavier_initializer())
            b1 = tf.Variable(tf.constant(0.1, shape=[256]), name="b31")
            self.output = tf.nn.xw_plus_b(self.h_drop, W1, b1, name="output3")
            self.output = tf.layers.batch_normalization(self.output, training=is_training)
            self.output = tf.nn.relu(self.output)
            self.output = tf.nn.dropout(self.output, self.keep_prob)
            W2 = tf.get_variable(
                "W32",
                shape=[256, num_classes[2]],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[num_classes[2]]), name="b32")
            l2_loss += tf.nn.l2_loss(W1)
            l2_loss += tf.nn.l2_loss(b1)
            self.scores3 = tf.nn.xw_plus_b(self.output, W2, b2, name="scores")
            self.predictions3 = tf.argmax(self.scores3, 1, name="predictions")
        # Calculate mean cross-entropy loss
        
        with tf.name_scope("loss"):
            losses1 = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores1, labels=self.input_y1)
            losses2 = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores2, labels=self.input_y2)
            losses3 = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores3, labels=self.input_y3)
            self.loss = 0.1 * tf.reduce_mean(losses1) + 0.2 * tf.reduce_mean(losses2) + 0.7*tf.reduce_mean(losses3) + l2_reg_lambda * l2_loss
            tf.summary.scalar('loss1',tf.reduce_mean(losses1))
            tf.summary.scalar('loss2',tf.reduce_mean(losses2))
            tf.summary.scalar('loss3',tf.reduce_mean(losses3))
            tf.summary.scalar('loss',self.loss)
        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions1 = tf.equal(self.predictions1, tf.argmax(self.input_y1, 1))
            correct_predictions2 = tf.equal(self.predictions2, tf.argmax(self.input_y2, 1))
            correct_predictions3 = tf.equal(self.predictions3, tf.argmax(self.input_y3, 1))
#           单独的准确率
            self.accuracy1 = tf.reduce_mean(tf.cast(correct_predictions1, "float"), name="accuracy1")
            self.accuracy2 = tf.reduce_mean(tf.cast(correct_predictions2, "float"), name="accuracy2")
            self.accuracy3 = tf.reduce_mean(tf.cast(correct_predictions3, "float"), name="accuracy3")
#           一起的准确率
            self.acc = tf.reduce_mean(tf.cast(correct_predictions1, "float")*tf.cast(correct_predictions2, "float")*tf.cast(correct_predictions3, "float"))
            tf.summary.scalar('acc1',self.accuracy1)
            tf.summary.scalar('acc2',self.accuracy2)
            tf.summary.scalar('acc3',self.accuracy3)
            tf.summary.scalar('acc',self.acc)
            
    def gru_cell(self):
        with tf.name_scope('gru_cell'):
            cell = rnn.GRUCell(self.hidden_size, reuse=tf.get_variable_scope().reuse)
        return rnn.DropoutWrapper(cell, output_keep_prob=self.keep_prob)

    def bi_gru(self, inputs):
        """build the bi-GRU network. 返回个所有层的隐含状态。"""
        cells_fw = [self.gru_cell() for _ in range(self.n_layer)]
        cells_bw = [self.gru_cell() for _ in range(self.n_layer)]
        initial_states_fw = [cell_fw.zero_state(self.batch_size, tf.float32) for cell_fw in cells_fw]
        initial_states_bw = [cell_bw.zero_state(self.batch_size, tf.float32) for cell_bw in cells_bw]
        outputs, _, _ = rnn.stack_bidirectional_dynamic_rnn(cells_fw, cells_bw, inputs,
                                                            initial_states_fw=initial_states_fw,
                                                            initial_states_bw=initial_states_bw, dtype=tf.float32)
        return outputs
    
    def bigru_inference(self, X_inputs):
#         inputs = tf.nn.embedding_lookup(self.title_embedding, X_inputs)
        output_bigru = self.bi_gru(X_inputs)
        output_att = self.task_specific_attention(output_bigru, self.hidden_size*2)
        return output_att
    
    def task_specific_attention(self, inputs, output_size,
                                initializer=layers.xavier_initializer(),
                                activation_fn=tf.tanh, scope=None):
        """
        Performs task-specific attention reduction, using learned
        attention context vector (constant within task of interest).
        Args:
            inputs: Tensor of shape [batch_size, units, input_size]
                `input_size` must be static (known)
                `units` axis will be attended over (reduced from output)
                `batch_size` will be preserved
            output_size: Size of output's inner (feature) dimension
        Returns:
           outputs: Tensor of shape [batch_size, output_dim].
        """
#         assert len(inputs.get_shape()) == 3 and inputs.get_shape()[-1].value is not None
        with tf.variable_scope(scope or 'attention') as scope:
            # u_w, attention 向量
            attention_context_vector = tf.get_variable(name='attention_context_vector', shape=[output_size],
                                                       initializer=initializer, dtype=tf.float32)
            # 全连接层，把 h_i 转为 u_i ， shape= [batch_size, units, input_size] -> [batch_size, units, output_size]
            input_projection = layers.fully_connected(inputs, output_size, activation_fn=activation_fn, scope=scope)
            # 输出 [batch_size, units]
            vector_attn = tf.reduce_sum(tf.multiply(input_projection, attention_context_vector), axis=2, keep_dims=True)
            attention_weights = tf.nn.softmax(vector_attn, dim=1)
            tf.summary.histogram('attention_weigths', attention_weights)
            weighted_projection = tf.multiply(inputs, attention_weights)
            outputs = tf.reduce_sum(weighted_projection, axis=1)
            return outputs  # 输出 [batch_size, hidden_size*2]

In [23]:

# batch 生成函数
import math
def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
  
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = np.min(((batch_num + 1) * batch_size, data_size))
            yield shuffled_data[start_index:end_index]
            

## 参数

In [24]:
# 特征数这里是第三级标签的特征数
max_features = 1192

# 句子填充的长度
sequence_len = 20

# batch 大小
batch_size = 256 

# 迭代次数
epochs = 100

is_training = True

# 词向量长度
embedding_dims = 24

# gru  的filters
num_filters = 32

# filter 的大小
filter_size = [1, 2, 3, 4, 5]

# 三个标签的类数
num_classes = [22, 191, 1192]

In [25]:
import tensorflow as tf
import numpy as np
import os
import time
import datetime
from tensorflow.contrib import learn



FLAGS = tf.flags.FLAGS

def train(x_train, y_train, x_dev, y_dev):
    # Training
    # ==================================================

    with tf.Graph().as_default():
        session_conf = tf.ConfigProto()
        sess = tf.Session(config=session_conf)
        with sess.as_default():
            cnn = GruModel(sequence_length = sequence_len, 
                          num_classes = num_classes, 
                          embedding_size = embedding_dims, 
                          num_filters = num_filters)

            # Define Training procedure

            global_step = tf.Variable(0, name="global_step", trainable=False)
#           学习率衰减
            learning_rate = tf.train.exponential_decay(
                0.01,
                global_step,
                x_train.shape[0] / batch_size,
                0.99,
                staircase=True,)
            optimizer = tf.train.AdamOptimizer(learning_rate)
            grads_and_vars = optimizer.compute_gradients(cnn.loss)
            train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

            # Initialize all variables
            sess.run(tf.global_variables_initializer())
            
            merged = tf.summary.merge_all()
            
            writer = tf.summary.FileWriter("fwwb",sess.graph)
            acc_sum = np.array([0.0,0.0,0.0,0.0])
            loss_sum = 0
            
            def train_step(x_batch, y_batch1, y_batch2, y_batch3, loss_sum, acc_sum, interal = 10):
  
                """
                A single training step
                """
                feed_dict = {
                  cnn.input_x: x_batch,
                  cnn.input_y1: y_batch1,
                  cnn.input_y2: y_batch2,
                  cnn.input_y3: y_batch3,
                  cnn.keep_prob: 0.9
                }

#         三个标签单独的准确率， 一起的准确率共四个准确率
                rs, _, step,  loss, accuracy1, accuracy2, accuracy3, acc = sess.run(
                    [merged, train_op, global_step, cnn.loss, cnn.accuracy1, cnn.accuracy2, cnn.accuracy3, cnn.acc],
                    feed_dict)
                time_str = datetime.datetime.now().isoformat()
                acc_sum += [accuracy1, accuracy2, accuracy3, acc]
                loss_sum += loss
                if step % interal == 0:
                    print("{}: step {}, loss {:g}, acc {}".format(time_str, step, loss_sum/interal, acc_sum/interal))
                    loss_sum = 0
                    acc_sum = np.array([0.0,0.0,0.0,0.0])
                    writer.add_summary(rs, step)
                return loss_sum, acc_sum
            # 评估步骤
            def dev_step(x_batch, y_batch):
                """
                Evaluates model on a dev set
                """

                dev_batches = batch_iter(
                    list(zip(x_batch, y_batch[0], y_batch[1], y_batch[2])), batch_size, 1)
                step = 0
                loss = 0
                accuracy = np.array([0.0, 0.0, 0.0,0.0])
                for batch in dev_batches:
                    x_batch_dev, y_batch1_dev, y_batch2_dev, y_batch3_dev = zip(*batch)
                    feed_dict = {
                      cnn.input_x: x_batch_dev,
                      cnn.input_y1: y_batch1_dev,
                      cnn.input_y2: y_batch2_dev,
                      cnn.input_y3: y_batch3_dev,
                      cnn.keep_prob: 1
                    }
                    if len(x_batch_dev) < batch_size:
                        continue
                    step,  temp_loss, temp_accuracy1, temp_accuracy2, temp_accuracy3 , acc= sess.run(
                        [global_step,cnn.loss, cnn.accuracy1, cnn.accuracy2, cnn.accuracy3, cnn.acc],
                        feed_dict)
                    accuracy[0] += temp_accuracy1 * len(x_batch_dev)
                    accuracy[1] += temp_accuracy2 * len(x_batch_dev)
                    accuracy[2] += temp_accuracy3 * len(x_batch_dev)
                    accuracy[3] += acc * len(x_batch_dev)
                    loss += temp_loss * len(x_batch_dev)
                accuracy /= x_batch.shape[0]
                loss /= x_batch.shape[0]
                time_str = datetime.datetime.now().isoformat()                    

                print("{}: step {}, loss {:g}, acc {} ".format(time_str, step, loss, accuracy))


            # Generate batches
            batches = batch_iter(
                list(zip(x_train, y_train[0],y_train[1], y_train[2] )), batch_size, 200)
            # Training loop. For each batch...
            for batch in batches:
                x_batch, y_batch1, y_batch2, y_batch3 = zip(*batch)
                if len(x_batch) < batch_size:
                    continue
                loss_sum, acc_sum = train_step(x_batch, y_batch1, y_batch2, y_batch3,  loss_sum, acc_sum, 40)
                current_step = tf.train.global_step(sess, global_step)
                if current_step % 200 == 0:
                    print("\nEvaluation:")
                    dev_step(x_dev, y_dev)
                    print("")

In [ ]:
y_train = [train_y_label1_one_hot,train_y_label2_one_hot,train_y_label3_one_hot]
y_test = [test_y_label1_one_hot,test_y_label2_one_hot,test_y_label3_one_hot]
train(x_train, y_train,  x_test, y_test)

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
dim is deprecated, use axis instead
gru shape (256, 64)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

2019-02-27T22:51:24.542035: step 40, loss 3.88515, acc [0.44296875 0.36660156 0.28271484 0.21464844]
2019-02-27T22:51:27.180222: step 80, loss 2.6546, acc [0.63574219 0.53984375 0.45703125 0.37216797]
2019-02-27T22:51:29.735120: step 120, loss 2.15493, acc [0.7125     0.62724609 0.5515625  0.46113281]
2019-02-27T22:51:32.423441: step 160, loss 1.84675, acc [0.76240234 0.67314453 0.6        0.51855469]
2019-02-27T22:51:35.001651: step 200, loss 1.6767, acc [0.79023438 0.70976562 0.63398438 0.55751953]

Evaluation:
2019-02-27T22:51:44.265991: step 200, loss 1.55321, acc [0.81129053 0.72649641 0.65275456 0.58267677] 

2019-02-27T22:51:47.127417: st

2019-02-27T22:55:44.161610: step 2400, loss 0.589818, acc [0.92324219 0.87597656 0.84082031 0.78398437]

Evaluation:
2019-02-27T22:55:53.320753: step 2400, loss 0.482066, acc [0.93728038 0.89844728 0.86923485 0.82304358] 

2019-02-27T22:55:56.314115: step 2440, loss 0.584294, acc [0.92167969 0.88320312 0.84130859 0.784375  ]
2019-02-27T22:55:58.882405: step 2480, loss 0.599293, acc [0.92177734 0.87421875 0.83779297 0.78300781]
2019-02-27T22:56:01.974968: step 2520, loss 0.584687, acc [0.92294922 0.88300781 0.83925781 0.78740234]
2019-02-27T22:56:04.585417: step 2560, loss 0.596656, acc [0.92236328 0.88574219 0.83730469 0.78515625]
2019-02-27T22:56:07.128412: step 2600, loss 0.595483, acc [0.92470703 0.8796875  0.84101563 0.78730469]

Evaluation:
2019-02-27T22:56:16.355555: step 2600, loss 0.459585, acc [0.93941275 0.90322258 0.87347956 0.82739841] 

2019-02-27T22:56:18.883624: step 2640, loss 0.573913, acc [0.92548828 0.88164062 0.84423828 0.78740234]
2019-02-27T22:56:21.329402: step 2

2019-02-27T23:00:22.843580: step 4960, loss 0.260325, acc [0.95380859 0.93466797 0.91904297 0.87021484]
2019-02-27T23:00:25.288365: step 5000, loss 0.256194, acc [0.95771484 0.93457031 0.92158203 0.87324219]

Evaluation:
2019-02-27T23:00:34.456933: step 5000, loss 0.315307, acc [0.95704232 0.93461743 0.91620699 0.87788445] 

2019-02-27T23:00:36.971160: step 5040, loss 0.270538, acc [0.95126953 0.93544922 0.91826172 0.86728516]
2019-02-27T23:00:39.415450: step 5080, loss 0.263339, acc [0.9546875  0.93232422 0.91953125 0.86630859]
2019-02-27T23:00:41.873628: step 5120, loss 0.276506, acc [0.95332031 0.93076172 0.91640625 0.86308594]
2019-02-27T23:00:44.304029: step 5160, loss 0.265883, acc [0.95439453 0.93095703 0.91767578 0.86376953]
2019-02-27T23:00:46.756802: step 5200, loss 0.284855, acc [0.95195312 0.92949219 0.91503906 0.86396484]

Evaluation:
2019-02-27T23:00:55.930280: step 5200, loss 0.322517, acc [0.95660183 0.93350619 0.91327373 0.87489113] 

2019-02-27T23:00:58.457423: step 5

2019-02-27T23:05:01.388225: step 7520, loss 0.249936, acc [0.95742187 0.93847656 0.921875   0.87304688]
2019-02-27T23:05:03.897492: step 7560, loss 0.245883, acc [0.95849609 0.93525391 0.92148438 0.87158203]
2019-02-27T23:05:06.362172: step 7600, loss 0.251668, acc [0.95810547 0.93808594 0.92246094 0.87451172]

Evaluation:
2019-02-27T23:05:15.555485: step 7600, loss 0.269286, acc [0.96394998 0.94322698 0.93228484 0.89640501] 

2019-02-27T23:05:18.106458: step 7640, loss 0.243339, acc [0.95634766 0.94150391 0.92070312 0.87177734]
2019-02-27T23:05:20.582453: step 7680, loss 0.254674, acc [0.95751953 0.93671875 0.92246094 0.87275391]
2019-02-27T23:05:23.071381: step 7720, loss 0.26102, acc [0.95439453 0.93632812 0.91914063 0.86826172]
2019-02-27T23:05:25.533526: step 7760, loss 0.254933, acc [0.95986328 0.93535156 0.9203125  0.87138672]
2019-02-27T23:05:27.992698: step 7800, loss 0.257805, acc [0.96064453 0.93515625 0.92041016 0.87402344]

Evaluation:
2019-02-27T23:05:37.208934: step 7800

2019-02-27T23:09:40.935662: step 10080, loss 0.147891, acc [0.9671875  0.95605469 0.95419922 0.90771484]
2019-02-27T23:09:43.406239: step 10120, loss 0.156844, acc [0.97050781 0.95585937 0.95029297 0.90947266]
2019-02-27T23:09:45.883822: step 10160, loss 0.154063, acc [0.96621094 0.95878906 0.95195312 0.90830078]
2019-02-27T23:09:48.379635: step 10200, loss 0.153644, acc [0.96826172 0.95556641 0.95078125 0.90712891]

Evaluation:
2019-02-27T23:09:57.639963: step 10200, loss 0.269188, acc [0.96758402 0.95177647 0.93995335 0.91004014] 

2019-02-27T23:10:00.177556: step 10240, loss 0.163575, acc [0.96630859 0.95458984 0.94951172 0.90654297]
2019-02-27T23:10:02.653536: step 10280, loss 0.168967, acc [0.96787109 0.95439453 0.94648438 0.90380859]
2019-02-27T23:10:05.116147: step 10320, loss 0.169789, acc [0.96484375 0.95488281 0.94628906 0.90068359]
2019-02-27T23:10:07.574819: step 10360, loss 0.167355, acc [0.96708984 0.95166016 0.94755859 0.90429688]
2019-02-27T23:10:10.068414: step 10400, 

...

2019-02-27T23:32:32.564583: step 22840, loss 0.0890644, acc [0.97763672 0.97402344 0.97011719 0.93925781]
2019-02-27T23:32:34.986588: step 22880, loss 0.095193, acc [0.97636719 0.97197266 0.9703125  0.93564453]
2019-02-27T23:32:37.394648: step 22920, loss 0.0937053, acc [0.978125   0.97246094 0.96904297 0.93476563]
2019-02-27T23:32:39.809660: step 22960, loss 0.0908402, acc [0.97880859 0.97041016 0.97089844 0.93623047]
2019-02-27T23:32:42.219232: step 23000, loss 0.0901006, acc [0.97880859 0.97255859 0.97119141 0.93789062]

Evaluation:
2019-02-27T23:32:51.393750: step 23000, loss 0.286199, acc [0.97494219 0.96322918 0.95322808 0.93224479] 

2019-02-27T23:32:53.901528: step 23040, loss 0.0902961, acc [0.9765625  0.97197266 0.97001953 0.93427734]
2019-02-27T23:32:56.324988: step 23080, loss 0.0941721, acc [0.97685547 0.97041016 0.96962891 0.93349609]
2019-02-27T23:32:58.735549: step 23120, loss 0.0891249, acc [0.97783203 0.97158203 0.96982422 0.93505859]
2019-02-27T23:33:01.161353: step 

2019-02-27T23:36:58.956645: step 25400, loss 0.0598993, acc [0.98212891 0.97900391 0.97958984 0.95      ]

Evaluation:
2019-02-27T23:37:08.163524: step 25400, loss 0.312038, acc [0.97554285 0.96372974 0.95360851 0.93370641] 

2019-02-27T23:37:10.664853: step 25440, loss 0.0752818, acc [0.98134766 0.97832031 0.97490234 0.94736328]
2019-02-27T23:37:13.101207: step 25480, loss 0.0735017, acc [0.9828125  0.978125   0.97460938 0.94648438]
2019-02-27T23:37:15.520200: step 25520, loss 0.066892, acc [0.98066406 0.97617188 0.97763672 0.94736328]
2019-02-27T23:37:17.934732: step 25560, loss 0.0668307, acc [0.98291016 0.9796875  0.97705078 0.94951172]
2019-02-27T23:37:20.353827: step 25600, loss 0.0724016, acc [0.98105469 0.97666016 0.97724609 0.94736328]

Evaluation:
2019-02-27T23:37:29.482263: step 25600, loss 0.303143, acc [0.97554285 0.96369971 0.95301785 0.93308573] 

2019-02-27T23:37:31.998980: step 25640, loss 0.0677953, acc [0.98222656 0.97695312 0.97929687 0.94960937]
2019-02-27T23:37:34

2019-02-27T23:41:31.870236: step 27920, loss 0.0916511, acc [0.97861328 0.97177734 0.97275391 0.93828125]
2019-02-27T23:41:34.297659: step 27960, loss 0.0809425, acc [0.97949219 0.97236328 0.97392578 0.93955078]
2019-02-27T23:41:36.742497: step 28000, loss 0.0876374, acc [0.97949219 0.97324219 0.96914062 0.93613281]

Evaluation:
2019-02-27T23:41:45.898117: step 28000, loss 0.297008, acc [0.97625364 0.96412017 0.95383876 0.93473756] 

2019-02-27T23:41:48.405894: step 28040, loss 0.0911461, acc [0.97900391 0.97128906 0.9703125  0.93642578]
2019-02-27T23:41:50.834312: step 28080, loss 0.0820273, acc [0.97939453 0.97490234 0.971875   0.94091797]
2019-02-27T23:41:53.381277: step 28120, loss 0.0566547, acc [0.98369141 0.98134766 0.98232422 0.95742187]
2019-02-27T23:41:55.802750: step 28160, loss 0.0564224, acc [0.98242188 0.98017578 0.98154297 0.95439453]
2019-02-27T23:41:58.205375: step 28200, loss 0.0551958, acc [0.9828125  0.97919922 0.98134766 0.95371094]

Evaluation:
2019-02-27T23:42:07

2019-02-27T23:46:05.307081: step 30440, loss 0.0692909, acc [0.98271484 0.97558594 0.97773438 0.94882813]
2019-02-27T23:46:07.726123: step 30480, loss 0.0628358, acc [0.98330078 0.9796875  0.97978516 0.95263672]
2019-02-27T23:46:10.159453: step 30520, loss 0.0639458, acc [0.98417969 0.98046875 0.97744141 0.95185547]
2019-02-27T23:46:12.627551: step 30560, loss 0.0759303, acc [0.97978516 0.97519531 0.97695312 0.94501953]
2019-02-27T23:46:15.057456: step 30600, loss 0.0639005, acc [0.98183594 0.97871094 0.97763672 0.94824219]

Evaluation:
2019-02-27T23:46:24.247848: step 30600, loss 0.323103, acc [0.97553284 0.96387991 0.95413909 0.934237  ] 

2019-02-27T23:46:26.744219: step 30640, loss 0.0820513, acc [0.98183594 0.97685547 0.97382813 0.94570312]
2019-02-27T23:46:29.181069: step 30680, loss 0.0706101, acc [0.97851562 0.97792969 0.97695312 0.94619141]
2019-02-27T23:46:31.609538: step 30720, loss 0.0687417, acc [0.98076172 0.97871094 0.97753906 0.94980469]
2019-02-27T23:46:34.043857: step

2019-02-27T23:50:31.441023: step 33000, loss 0.0496808, acc [0.98623047 0.98212891 0.98349609 0.959375  ]

Evaluation:
2019-02-27T23:50:40.619070: step 33000, loss 0.323782, acc [0.97749502 0.96586211 0.95637157 0.93900229] 

2019-02-27T23:50:43.119904: step 33040, loss 0.0499937, acc [0.98691406 0.98378906 0.98359375 0.96025391]
2019-02-27T23:50:45.531457: step 33080, loss 0.0586996, acc [0.98476562 0.98164063 0.98193359 0.95664063]
2019-02-27T23:50:47.948964: step 33120, loss 0.0555299, acc [0.98642578 0.98193359 0.98222656 0.95761719]
2019-02-27T23:50:50.407639: step 33160, loss 0.0569566, acc [0.9828125  0.98144531 0.98125    0.95644531]
2019-02-27T23:50:52.843001: step 33200, loss 0.0536696, acc [0.98535156 0.98330078 0.98261719 0.95996094]

Evaluation:
2019-02-27T23:51:02.009088: step 33200, loss 0.322639, acc [0.97646387 0.96589214 0.95676201 0.93873199] 

2019-02-27T23:51:04.509921: step 33240, loss 0.0513304, acc [0.98378906 0.98349609 0.98271484 0.95820313]
2019-02-27T23:51:0

In [ ]:
test_y_label3_one_hot.shape

In [ ]:
train_y_label3_one_hot.shape